# RDClassify with Fastai and PyTorch 

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import rdflib
import requests
from fastai.text import *
import sys
sys.path.append("../")

Downloads the latest export files from Sinopia's stage environment

In [3]:
dev_url = "https://sinopia-exports-development.s3-us-west-2.amazonaws.com/sinopia_export_all_2020-04-05T00:01:17.464Z.zip"
stage_url = "https://sinopia-exports-staging.s3-us-west-2.amazonaws.com/sinopia_export_all_2020-04-05T00:02:01.951Z.zip"
prod_url = "https://sinopia-exports-production.s3-us-west-2.amazonaws.com/sinopia_export_all_2020-04-05T00:01:56.669Z.zip"

In [14]:
class RDFItem(Text):
    
    def __init__(self, url):
        self.text = url
        print(f"{self}")

In [19]:
item = RDFItem(str(rdflib.RDF.type))


http://www.w3.org/1999/02/22-rdf-syntax-ns#type


In [20]:
dir(item)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_tfms',
 'show',
 'text']

In [21]:
item.text

'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'

In [28]:
class RDFList(TextList):
    
    def __init__(self, **kwargs):
        self.items = kwargs.get('items', [])
        print(f"{self}")

In [29]:
rdf_list = RDFList()

AttributeError: 'RDFList' object has no attribute 'path'

In [6]:
# Using a tabular approach
import pandas as pd
import requests
from fastai.tabular import *

In [5]:
from rdf_classify.data_loader import from_zipfile

In [1]:
staging = from_zipfile("/Users/jpnelson/Downloads/sinopia_export_all_2020-04-12T00_02_02.117Z.zip")

NameError: name 'from_zipfile' is not defined

In [8]:
len(staging)


3490

In [2]:
import rdflib

In [70]:
SUBJ_RT = """SELECT ?subj ?resource_template 
WHERE { ?subj <http://sinopia.io/vocabulary/hasResourceTemplate> ?resource_template }"""

def subject_to_str(subject):
    if isinstance(subject, rdflib.BNode):
        subject = subject.skolemize(authority='https://trellis.stage.sinopia.io/repository/')
    return str(subject)
    

def create_series(graph):
    rt_query = graph.query(SUBJ_RT)
    for subj, rt in rt_query:                
        subject = { 'subject': subject_to_str(subj), 
                    'resource_template': str(rt),
                    f"{rdflib.RDF.type}": []
                  }
        
                    
def process_predicates(graph: rdflib.Graph, subject: dict) -> dict:
     for predicate in graph.predicates(subject=subj):
        pred_str = str(predicate)
        if predicate == rdflib.RDF.type:
            rdf_class = graph.get(subject=subj, predicate=predicate)
            subject[pred_str].append(str(rdf_class))
            continue
        if pred_str in subject:
            subject[pred_str] += 1
        else:
            subject[pred_str] = 1
    return subject
                    
def to_dataframe(graphs: List[rdflib.Graph]) -> pd.DataFrame: 
    raw_data = []
    for i,graph in enumerate(graphs):
        if not i%10:
            print(".", end="")
        if not i%100:
            print(f"{i}", end="")
        rt_query = graph.query(SUBJ_RT)

           
                
            raw_data.append(subject)
    return pd.DataFrame(data=raw_data)

In [3]:
print(rdflib.RDF.type)

http://www.w3.org/1999/02/22-rdf-syntax-ns#type


In [71]:
staging_df = to_dataframe(staging)

.0..........100..........200..........300..........400..........500..........600..........700..........800..........900..........1000..........1100..........1200..........1300..........1400..........1500..........1600..........1700..........1800..........1900..........2000..........2100..........2200..........2300..........2400..........2500..........2600..........2700..........2800..........2900..........3000..........3100..........3200..........3300..........3400........

In [72]:
len(staging_df)

3761

In [73]:
staging_df.head()

,subject,resource_template,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/ns/prov#wasGeneratedBy,http://sinopia.io/vocabulary/hasResourceTemplate,http://id.loc.gov/ontologies/bibframe/hasContribution,http://id.loc.gov/ontologies/bibframe/title,http://id.loc.gov/ontologies/bibframe/agent,http://id.loc.gov/ontologies/bibframe/adminMetadata,http://id.loc.gov/ontologies/bibframe/carrier,...,http://id.loc.gov/ontologies/bibframe/RegionalEncoding,http://id.loc.gov/ontologies/bibframe/RecordingMedium,http://id.loc.gov/ontologies/bibframe/FileType,http://id.loc.gov/ontologies/bibframe/RecordingMethod,http://id.loc.gov/ontologies/bibframe/subseriesStatement,http://id.loc.gov/ontologies/bibframe/subseriesEnumeration,http://id.loc.gov/ontologies/bibframe/coordinates,http://id.loc.gov/ontologies/bibframe/succeededBy,http://id.loc.gov/ontologies/bibframe/hasPart,http://id.loc.gov/ontologies/bibframe/relatedTo
0,https://trellis.stage.sinopia.io/repository/pe...,michelle:RT:quick,1,1.0,1,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://trellis.stage.sinopia.io/.well-known/g...,michelle:RT:bf2:Agents:PrimaryContribution,1,NaN,1,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://trellis.stage.sinopia.io/repository/pe...,ld4p:RT:bf2:Monograph:Instance,1,3.0,1,NaN,3.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://trellis.stage.sinopia.io/repository/pe...,ld4p:RT:bf2:Monograph:Instance,1,1.0,1,NaN,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://trellis.stage.sinopia.io/repository/pe...,ld4p:RT:bf2:Monograph:Work,1,3.0,1,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
data = (TabularList.from_df(staging_df,
                            
                            procs=[FillMissing])
        .databunch())

Exception: Your data is neither split nor labeled, can't turn it into a `DataBunch` yet.

In [69]:
data

TabularList (3761 items)
subject                                                       https://trellis.stage.sinopia.io/repository/pe...
resource_template                                                                             michelle:RT:quick
http://www.w3.org/1999/02/22-rdf-syntax-ns#type                                                               1
http://www.w3.org/ns/prov#wasGeneratedBy                                                                      1
http://sinopia.io/vocabulary/hasResourceTemplate                                                              1
                                                                                    ...                        
http://id.loc.gov/ontologies/bibframe/subseriesEnumeration                                                  NaN
http://id.loc.gov/ontologies/bibframe/coordinates                                                           NaN
http://id.loc.gov/ontologies/bibframe/succeededBy                              